In [1]:
import json
import random

import matplotlib.pyplot
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.collections as mcoll
import matplotlib.colors as mcolors
from matplotlib.widgets import Button
import json as js
from math import sin, cos, pi

TOLERANCE = 0.15

def dist(point1, point2):
    return np.sqrt(np.power(point1[0] - point2[0], 2) + np.power(point1[1] - point2[1], 2))

class _Button_callback(object):
    def __init__(self, scenes):
        self.i = 0
        self.scenes = scenes
        self.adding_points = False
        self.added_points = []
        self.adding_lines = False
        self.added_lines = []
        self.adding_rects = False
        self.added_rects = []

    def set_axes(self, ax):
        self.ax = ax

    def next(self, event):
        self.i = (self.i + 1) % len(self.scenes)
        self.draw(autoscaling = True)

    def prev(self, event):
        self.i = (self.i - 1) % len(self.scenes)
        self.draw(autoscaling = True)

    def add_point(self, event):
        self.adding_points = not self.adding_points
        self.new_line_point = None
        if self.adding_points:
            self.adding_lines = False
            self.adding_rects = False
            self.added_points.append(PointsCollection([]))

    def add_line(self, event):
        self.adding_lines = not self.adding_lines
        self.new_line_point = None
        if self.adding_lines:
            self.adding_points = False
            self.adding_rects = False
            self.added_lines.append(LinesCollection([]))

    def add_rect(self, event):
        self.adding_rects = not self.adding_rects
        self.new_line_point = None
        if self.adding_rects:
            self.adding_points = False
            self.adding_lines = False
            self.new_rect()

    def new_rect(self):
        self.added_rects.append(LinesCollection([]))
        self.rect_points = []

    def on_click(self, event):
        if event.inaxes != self.ax:
            return
        new_point = (event.xdata, event.ydata)
        if self.adding_points:
            self.added_points[-1].add_points([new_point])
            self.draw(autoscaling = False)
        elif self.adding_lines:
            if self.new_line_point is not None:
                self.added_lines[-1].add([self.new_line_point, new_point])
                self.new_line_point = None
                self.draw(autoscaling = False)
            else:
                self.new_line_point = new_point
        elif self.adding_rects:
            if len(self.rect_points) == 0:
                self.rect_points.append(new_point)
            elif len(self.rect_points) == 1:
                self.added_rects[-1].add([self.rect_points[-1], new_point])
                self.rect_points.append(new_point)
                self.draw(autoscaling = False)
            elif len(self.rect_points) > 1:
                if dist(self.rect_points[0], new_point) < (np.mean([self.ax.get_xlim(), self.ax.get_ylim()])*TOLERANCE):
                    self.added_rects[-1].add([self.rect_points[-1], self.rect_points[0]])
                    self.new_rect()
                else:
                    self.added_rects[-1].add([self.rect_points[-1], new_point])
                    self.rect_points.append(new_point)
                self.draw(autoscaling = False)

    def draw(self, autoscaling = True):
        if not autoscaling:
            xlim = self.ax.get_xlim()
            ylim = self.ax.get_ylim()
        self.ax.clear()
        for collection in (self.scenes[self.i].lines + self.added_lines + self.added_rects):
            self.ax.add_collection(collection.get_collection())
        for collection in (self.scenes[self.i].points + self.added_points):
            if len(collection.points) > 0:
                self.ax.scatter(*zip(*(np.array(collection.points))), **collection.kwargs)
        self.ax.autoscale(autoscaling)
        if not autoscaling:
            self.ax.set_xlim(xlim)
            self.ax.set_ylim(ylim)
        plt.draw()

In [2]:
class Scene:
    def __init__(self, points=[], lines=[]):
        self.points=points
        self.lines=lines

class PointsCollection:
    def __init__(self, points, **kwargs):
        self.points = points
        self.kwargs = kwargs

    def add_points(self, points):
        self.points = self.points + points

class LinesCollection:
    def __init__(self, lines, **kwargs):
        self.lines = lines
        self.kwargs = kwargs

    def add(self, line):
        self.lines.append(line)

    def get_collection(self):
        return mcoll.LineCollection(self.lines, **self.kwargs)

class Plot:
    def __init__(self, scenes = [Scene()], points = [], lines = [], json = None):
        if json is None:
            self.scenes = scenes
            if points or lines:
                self.scenes[0].points = points
                self.scenes[0].lines = lines
        else:
            self.scenes = [Scene([PointsCollection(pointsCol) for pointsCol in scene["points"]],
                                 [LinesCollection(linesCol) for linesCol in scene["lines"]])
                           for scene in js.loads(json)]

    def __configure_buttons(self):
        plt.subplots_adjust(bottom=0.2)
        ax_prev = plt.axes([0.6, 0.05, 0.15, 0.075])
        ax_next = plt.axes([0.76, 0.05, 0.15, 0.075])
        ax_add_point = plt.axes([0.44, 0.05, 0.15, 0.075])
        ax_add_line = plt.axes([0.28, 0.05, 0.15, 0.075])
        ax_add_rect = plt.axes([0.12, 0.05, 0.15, 0.075])
        b_next = Button(ax_next, 'Następny')
        b_next.on_clicked(self.callback.next)
        b_prev = Button(ax_prev, 'Poprzedni')
        b_prev.on_clicked(self.callback.prev)
        b_add_point = Button(ax_add_point, 'Dodaj punkt')
        b_add_point.on_clicked(self.callback.add_point)
        b_add_line = Button(ax_add_line, 'Dodaj linię')
        b_add_line.on_clicked(self.callback.add_line)
        b_add_rect = Button(ax_add_rect, 'Dodaj figurę')
        b_add_rect.on_clicked(self.callback.add_rect)
        return [b_prev, b_next, b_add_point, b_add_line, b_add_rect]

    def add_scene(self, scene):
        self.scenes.append(scene)

    def add_scenes(self, scenes):
        self.scenes = self.scenes + scenes

    def toJson(self):
        return js.dumps([{"points": [np.array(pointCol.points).tolist() for pointCol in scene.points],
                          "lines":[linesCol.lines for linesCol in scene.lines]}
                         for scene in self.scenes])

    def get_added_points(self):
        if self.callback:
            return self.callback.added_points
        else:
            return None

    def get_added_lines(self):
        if self.callback:
            return self.callback.added_lines
        else:
            return None

    def get_added_figure(self):
        if self.callback:
            return self.callback.added_rects
        else:
            return None

    def get_added_elements(self):
        if self.callback:
            return Scene(self.callback.added_points, self.callback.added_lines+self.callback.added_rects)
        else:
            return None

    def draw(self):
        plt.close()
        fig = plt.figure()
        self.callback = _Button_callback(self.scenes)
        self.widgets = self.__configure_buttons()
        ax = plt.axes(autoscale_on = False)
        self.callback.set_axes(ax)
        fig.canvas.mpl_connect('button_press_event', self.callback.on_click)
        plt.show()
        self.callback.draw()

# Rozwiązanie Lab3

### Funkcja do zadawaia zbiorów punktów

In [27]:
%matplotlib notebook

plot1 = Plot()
plot1.draw()

<IPython.core.display.Javascript object>

### Zapis podanego wcześniej wielokąta do pliku

In [28]:
%matplotlib notebook

plot2 = Plot([plot1.get_added_elements()])
with open("plots/number_of_plots.txt", 'r') as f:
    counter = int(f.readline())
counter += 1
with open("plots/number_of_plots.txt", 'w') as f:
    f.write(str(counter))
with open('plots/plot' + str(counter) + '.json', 'w') as file:
    file.write(plot2.toJson())
plot2.draw()

<IPython.core.display.Javascript object>

### Funkcja convert_js( zwraca tylko kolejne punkty)

In [3]:
def convert_js(f_name):
    fi = open(f_name)
    plot = json.load(fi)
    fi.close()
    scene = plot[0]
    sets_of_points = scene['points']
    first_set = sets_of_points[0]
    points = []
    for a, b in first_set:
        points.append((a, b))
    return points


### Funkcja do zmiany reprezentacji wielokąta

In [4]:
def change_to_graph(points):
    n = len(points)
    prev = [i for i in range(n)]
    after = [i for i in range(n)]
    prev[0] = n - 1
    after[0] = 1
    prev[-1] = n - 2
    after[-1] = 0
    for i in range(1, n - 1):
        prev[i] = i - 1
        after[i] = i + 1
    return prev, after



### Dodawanie boków wielokąta

In [5]:
def add_sides(points, prev, after):
    sides = []
    cur = after[0]
    while cur != 0:
        sides.append((points[prev[cur]], points[cur]))
        cur = after[cur]
    sides.append((points[0], points[prev[0]]))
    return sides

### Funkcje do rysowania danych wielokątów

In [6]:
%matplotlib notebook

def add_polygon_scene(scenes, plot_number):
    points = convert_js("plots/plot" + str(plot_number)+".json")
    prev, after = change_to_graph(points)
    sides = add_sides(points, prev, after)
    scenes.append(Scene(points=[PointsCollection(points)], lines=[LinesCollection(sides)]))

def draw_given_polygons(indices):
    scenes = []
    for ind in indices:
        add_polygon_scene(scenes, ind)
    return Plot(scenes=scenes)

def draw_all_polygons():
    with open("plots/number_of_plots.txt", 'r') as no_of_plots:
        how_many = int(no_of_plots.readline())

    scenes = []
    for i in range(1, how_many + 1):
        add_polygon_scene(scenes, i)
    return Plot(scenes=scenes)


In [31]:
%matplotlib notebook

plots1 = draw_all_polygons()
plots1.draw()

<IPython.core.display.Javascript object>

### Implementacja wyznacznika

In [8]:
epsilon = pow(10, -10)
def det(a, b, c):
    d =  (a[0]*b[1] + a[1]*c[0] + b[0]*c[1] -
            c[0]*b[1] - a[0]*c[1] - b[0]*a[1])
    if abs(d) < epsilon:
        return 0
    return d

## Sprawdzanie monotoniczności

In [9]:
def check_monotonic(points, prev, after):
    for i in range(len(points)):
        if (not (points[prev[i]][1] - points[i][1])*(points[after[i]][1] - points[i][1]) < 0) and det(points[prev[i]], points[after[i]], points[i]) > 0:
            return False
    return True

## Klasyfikacja punktów

In [10]:
%matplotlib notebook

def color_points(scenes, points, prev, after, lines):
    beg_points = []
    end_points = []
    con_points = []
    div_points = []
    reg_points = []
    for i in range(len(points)):
        if (points[prev[i]][1] - points[i][1])*(points[after[i]][1] - points[i][1]) < 0:
            reg_points.append(points[i])
        else:
            d = det(points[prev[i]], points[after[i]], points[i])
            if points[prev[i]][1] < points[i][1]:
                if d < 0:
                    beg_points.append(points[i])
                else:
                    div_points.append(points[i])
            else:
                if d < 0:
                    end_points.append(points[i])
                else:
                    con_points.append(points[i])
    scenes.append(Scene(points=[PointsCollection(beg_points, color='green'), PointsCollection(end_points, color='red'), PointsCollection(con_points, color='purple'), PointsCollection(div_points), PointsCollection(reg_points, color='orange')], lines=[LinesCollection(lines, color='grey')]))


### Rysowanie wielokątów po klasyfikacji

In [11]:
%matplotlib notebook

def classify_given_polygons(indices):
    scenes = []
    for ind in indices:
        polygon = convert_js("plots/plot" + str(ind)+ ".json")
        prev, after = change_to_graph(polygon)
        sides = add_sides(polygon, prev, after)
        color_points(scenes, polygon, prev, after, sides)
    return Plot(scenes=scenes)

def classify_all_polygons():
    with open("plots/number_of_plots.txt", 'r') as no_of_plots:
        how_many = int(no_of_plots.readline())

    scenes = []
    for i in range(1, how_many + 1):
        polygon = convert_js("plots/plot" + str(i)+ ".json")
        prev, after = change_to_graph(polygon)
        sides = add_sides(polygon, prev, after)
        color_points(scenes, polygon, prev, after, sides)
    return Plot(scenes=scenes)

In [29]:
%matplotlib notebook

plots2 = classify_all_polygons()
plots2.draw()

<IPython.core.display.Javascript object>

## Triangulacja wielokąta monotonicznego

In [78]:
%matplotlib notebook

def triangulate_monotonic(points, prev, after, sides):
    def copyp(tab):
        return [points[tab[j]] for j in range(len(tab))]

    def copyl(tab):
        return [(points[tab[j][0]], points[tab[j][1]]) for j in range(len(tab))]

    def add_scene1():
        scenes.append(Scene(points=[PointsCollection(points), PointsCollection(copyp(left), color='green'), PointsCollection(copyp(right), color='red'), PointsCollection([points[first], points[last]], color='orange')], lines=[LinesCollection(sides), LinesCollection(copyl(left_lines), color='green'), LinesCollection(copyl(right_lines), color='red')]))

    def add_scene2():
        scenes.append(Scene(points=[PointsCollection(points)], lines=[LinesCollection(sides), LinesCollection(copyl(triangles_lines), color= 'red')]))

    def merge():
        i = 0
        j = 0
        while i < len(left) and j < len(right):
            if points[left[i]][1] >= points[right[j]][1]:
                merged.append((left[i], True))
                i += 1
            else:
                merged.append((right[j], False))
                j += 1

        while i < len(left):
            merged.append((left[i], True))
            i += 1

        while j < len(right):
            merged.append((right[j], False))
            j += 1

    scenes = []
    n = len(points)
    scenes.append(Scene(points=[PointsCollection(points)], lines=[LinesCollection(sides)]))
    first = 0
    last = 0
    for i in range(1, n):
        if points[i][1] > points[first][1]:
            first = i
        if points[i][1] < points[last][1]:
            last = i

    left, right, left_lines, right_lines = [], [], [], []
    add_scene1()
    cur = after[first]

    while cur != last:
        left.append(cur)
        left_lines.append((cur, prev[cur]))
        add_scene1()
        cur = after[cur]

    left_lines.append((cur, prev[cur]))
    add_scene1()
    cur = after[last]

    while cur != first:
        right.append(cur)
        right_lines.append((cur, prev[cur]))
        add_scene1()
        cur = after[cur]

    right_lines.append((cur, prev[cur]))
    add_scene1()
    right.reverse()
    merged = []
    merge()
    merged.append((last, merged[-1][1]))
    stack = [(first, True), merged[0]]
    triangles_lines = []
    add_scene2()

    for x, side in merged[1:]:
        if stack[-1][1] != side:
            top = stack.pop()
            if after[x] != top[0] and prev[x] != top[0]:
                triangles_lines.append((x, top[0]))
                add_scene2()
            while stack:
                if stack[-1][1] != side and after[x] != stack[-1][0] and prev[x] != stack[-1][0]:
                    triangles_lines.append((x, stack[-1][0]))
                    add_scene2()
                stack.pop()
            stack.append(top)
        else:
            while len(stack) > 1:
                d = det(points[stack[-2][0]], points[x], points[stack[-1][0]])
                if d*(2*side - 1) < 0 and after[x] != stack[-2][0] and prev[x] != stack[-2][0]:
                    triangles_lines.append((stack[-2][0], x))
                    add_scene2()
                else:
                    break
                stack.pop()
        stack.append((x, side))
    return Plot(scenes=scenes), scenes[-1]

## Wizualizacja Triangulacji

### Wyświetlenie wyników triangulacji wszystkich wielokątów monotonicznych( w kolejnych scenach kolejne wielokąty)

In [73]:
%matplotlib notebook

def triangulate_given_polygon(ind):
    polygon = convert_js("plots/plot" + str(ind)+ ".json")
    prev, after = change_to_graph(polygon)
    sides = add_sides(polygon, prev, after)
    if not check_monotonic(polygon, prev, after):
        return classify_given_polygons([ind])
    plts, last_scene = triangulate_monotonic(polygon, prev, after, sides)
    return plts

def triangulate_all_polygons():
    with open("plots/number_of_plots.txt", 'r') as no_of_plots:
        how_many = int(no_of_plots.readline())

    scenes = []
    for i in range(1, how_many + 1):
        polygon = convert_js("plots/plot" + str(i)+ ".json")
        prev, after = change_to_graph(polygon)
        sides = add_sides(polygon, prev, after)
        if not check_monotonic(polygon, prev, after):
            continue
        plts, last_scene = triangulate_monotonic(polygon, prev, after, sides)
        scenes.append(last_scene)

    return Plot(scenes=scenes)


In [79]:
%matplotlib notebook

pl = triangulate_all_polygons()
pl.draw()

In [56]:
%matplotlib notebook

pl = triangulate_given_polygon(8)
pl.draw()